In [1]:
#import tempfile
import os
import datetime
import ocgis 
from ocgis.util.shp_process import ShpProcess
from ocgis.util.shp_cabinet import ShpCabinetIterator
from cdo import *   # python version
cdo = Cdo()

In [2]:
# define the environment
if 'obelix' in os.uname()[1] or 'asterix' in os.uname()[1]:
    HOME = os.getenv('HOME')
    OUT_PATH = '/home/estimr2/nhempelmann/data/extremoscop/'
    CACHE = '/home/estimr2/nhempelmann/cache/'
    IN_DIR = '/home/estimr2/EUROCORDEX/EOBS_v11/'
else:
    HOME = os.getenv('HOME')
    OUT_DIR = HOME+'/data/extremoscop/'

In [3]:
# define ocgis environment

ocgis.env.OVERWRITE = True
ocgis.env.USE_CFUNITS = True
SHP_DIR = os.path.join(HOME + '/birdhouse/flyingpigeon/flyingpigeon/processes/shapefiles/FRA/')
ocgis.env.DIR_SHPCABINET = SHP_DIR
#ocgis.env.DIR_OUTPUT = OUT_DIR
sc = ocgis.ShpCabinet()
geoms = 'regions-2016'
sci = ShpCabinetIterator(geoms)

In [4]:
ref_19611900 = [datetime.datetime(1961,1,1),datetime.datetime(1990,12,31)]
ref_19712000 = [datetime.datetime(1971,1,1),datetime.datetime(2000,12,31)]
ref_19762005 = [datetime.datetime(1976,1,1),datetime.datetime(2005,12,31)]
vars = ['pr', 'tas', 'tasmin', 'tasmax']
indices = ['R20mm']

In [5]:
# pr_file =  os.path.join(CACHE, 'pr_EOBS_0.22_rot_v11_19500101-20141231_COEDEXformat.nc')
# pr_file =  os.path.join(IN_DIR, 'pr_EOBS_0.22_rot_v11_19500101-20141231.nc')

In [6]:
# pr_file

In [7]:
# define ocgis environment

ocgis.env.OVERWRITE = True
ocgis.env.USE_CFUNITS = True
SHP_DIR = os.path.join(HOME + '/birdhouse/flyingpigeon/flyingpigeon/processes/shapefiles/FRA/')
ocgis.env.DIR_SHPCABINET = SHP_DIR
#ocgis.env.DIR_OUTPUT = OUT_DIR
sc = ocgis.ShpCabinet()
geoms = 'regions-2016'
sci = ShpCabinetIterator(geoms)

calc = [{'func': 'icclim_%s' % indices[0], 'name': indices[0]}]
calc_group = ['year'] # [[12, 1, 2], 'unique'

In [10]:
#for nc in pr_file:
dimension_map = {'X': {'variable': 'longitude', 'dimension': 'x', 'pos': 2},
                 'Y': {'variable': 'latitude', 'dimension': 'y', 'pos': 1},
                 'T': {'variable': 'time',  'dimension': 'time', 'pos': 0}} #, 'bounds': 'time_bnds'

nc = 'pr_EOBS_0.22_rot_v11_19500101-20141231.nc'
rd = ocgis.RequestDataset(os.path.join(IN_DIR, nc), 'pr', dimension_map = dimension_map, 
                          conform_units_to = "kg m-2 s-1")


In [13]:
for indice in indices:
    for ugid in [1, 2, 3, 4, 5, 6, 7, 11,  13, 14, 15, 16, 17]: #  8, 9, 10, 12,
        scenario = 'eobs'
        prefix = nc.replace('pr', indice).replace('.nc', '')
            # create foulder structure and define output dir:
            
        if not os.path.exists(os.path.join(OUT_PATH , 'timeseries',  indice ,scenario , str(ugid))):
            os.makedirs(os.path.join(OUT_PATH , 'timeseries', indice ,scenario , str(ugid)))
        OUT_DIR = os.path.join(OUT_PATH , 'timeseries', indice ,scenario , str(ugid))
            
        if not os.path.exists(os.path.join(OUT_PATH , 'polygons',  indice ,scenario, str(ugid))):
            os.makedirs(os.path.join(OUT_PATH , 'polygons', indice ,scenario, str(ugid)))
        ocgis.env.DIR_OUTPUT = os.path.join(OUT_PATH , 'polygons', indice ,scenario, str(ugid))
            
        output = os.path.join(OUT_DIR,prefix+'.nc') # cdo output
            
        # if not os.path.exists(output):
        try:

            rd = ocgis.RequestDataset(os.path.join(IN_DIR, nc), 'pr',  dimension_map = dimension_map)
            # conform_units_to = "kg m-2 s-1",

            geom_nc = ocgis.OcgOperations(dataset=rd, geom=geoms, output_format='nc', 
                                              select_ugid = [ugid], prefix=prefix , 
                                              add_auxiliary_files=False ,calc=calc, 
                                              calc_grouping=calc_group).execute() #

            cdo.fldmean (input = str(geom_nc) , output = output )

            print ugid, prefix
        except Exception as e:
            print 'failed for file : %s \n %s ' % (prefix, e )
        #else: 
        #    print '%s \t %s already processed' % (ugid, prefix) 

1 R20mm_EOBS_0.22_rot_v11_19500101-20141231
2 R20mm_EOBS_0.22_rot_v11_19500101-20141231
3 R20mm_EOBS_0.22_rot_v11_19500101-20141231
4 R20mm_EOBS_0.22_rot_v11_19500101-20141231
5 R20mm_EOBS_0.22_rot_v11_19500101-20141231
6 R20mm_EOBS_0.22_rot_v11_19500101-20141231
7 R20mm_EOBS_0.22_rot_v11_19500101-20141231
11 R20mm_EOBS_0.22_rot_v11_19500101-20141231
13 R20mm_EOBS_0.22_rot_v11_19500101-20141231
14 R20mm_EOBS_0.22_rot_v11_19500101-20141231
15 R20mm_EOBS_0.22_rot_v11_19500101-20141231
16 R20mm_EOBS_0.22_rot_v11_19500101-20141231
17 R20mm_EOBS_0.22_rot_v11_19500101-20141231


In [188]:
#geom_nc
#output
#cdo.timstd(input = str(output), output = '/home/estimr2/nhempelmann/data/extremoscop/test.nc')
#output 

In [166]:
import pandas as pd
import netCDF4
from numpy import squeeze
import numpy as np

file = '/home/estimr2/nhempelmann/data/extremoscop/timeseries/R20mm/eobs/1/R20mm_EOBS_0.22_rot_v11_19500101-20141231.nc'

nc = netCDF4.Dataset(file, mode="a")
h =  squeeze(nc.variables[indice])
times = nc.variables['time']
jd = netCDF4.num2date(times[:],times.units)
hs = pd.Series(h,index=jd)

In [190]:
def add_statistics2(nc_url, var):
    """ add statistic variables to a given netCDF file for a given variable. 
    The statisitcs is calculated as a running 30 Year moving time window"""
 
    import pandas as pd
    import netCDF4
    from numpy import squeeze
    import numpy as np 

    nc = netCDF4.Dataset(nc_url, mode="a")
    h =  squeeze(nc.variables[var])
    times = nc.variables['time']
    jd = netCDF4.num2date(times[:],times.units)
    hs = pd.Series(h,index=jd)
    
    showyear = cdo.showyear(input = file )
    years = [int(i) for i in showyear[0].split(' ')]

    nc.createDimension('ref_period', len(period))
    
    ref_period = nc.createVariable(varname= 'ref_period', datatype = 'i', dimensions='ref_period')
    ref_median = nc.createVariable(varname= 'ref_median', datatype = 'float', dimensions='ref_period')
    ref_mean = nc.createVariable(varname= 'ref_mean', datatype = 'float', dimensions='ref_period')
    ref_std = nc.createVariable(varname= 'ref_std', datatype = 'float', dimensions='ref_period')

    ref_period_att = {'standard_name':'ref_period',
                  'long_name':'start year of reference period',
                  'units':'year' }
    ref_median_att = {'standard_name':'ref_median',
                  'long_name':'median for reference period'}
    ref_mean_att = {'standard_name':'ref_mean',
                  'long_name':'mean for reference period'}
    ref_std_att = {'standard_name':'ref_std',
                  'long_name':'standart deviation for reference period'}


    ref_period.setncatts(ref_period_att)
    ref_std.setncatts(ref_std_att)
    ref_mean.setncatts(ref_mean_att)
    ref_median.setncatts(ref_median_att)


    ref_periods = []
    ref_stds = []
    ref_medians = []
    ref_means = []

    for start in years[:-29]:
        end = start + 30
        ref_periods.append(start)
        ref_stds.append(hs[str(start):str(end)].std())
        ref_medians.append(hs[str(start):str(end)].median())
        ref_means.append(hs[str(start):str(end)].mean())


    ref_period[:] = ref_periods
    ref_median[:] = ref_medians
    ref_mean[:] = ref_means
    ref_std[:] = ref_stds

    nc.close()

In [197]:
import sys
sys.path.append('/home/users/nhempel/birdhouse/flyingpigeon/flyingpigeon')

from timeseries import add_statistics

In [ ]:
file = '/home/estimr2/nhempelmann/data/extremoscop/timeseries/R20mm/eobs/2/R20mm_EOBS_0.22_rot_v11_19500101-20141231.nc'
print file , indice

In [198]:
add_statistics?

In [189]:
file

'/home/estimr2/nhempelmann/data/extremoscop/timeseries/R20mm/eobs/2/R20mm_EOBS_0.22_rot_v11_19500101-20141231.nc'

In [129]:
showyear = cdo.showyear(input = file )
years = [int(i) for i in showyear[0].split(' ')]

In [153]:
ref_periods = []
ref_stds = []
ref_medians = []
ref_means = []

for start in years[:-29]:
    end = start + 30
    ref_periods.append(start)
    ref_stds.append(hs[str(start):str(end)].std())
    ref_medians.append(hs[str(start):str(end)].median())
    ref_means.append(hs[str(start):str(end)].mean())

In [ ]:
nc.createDimension('ref_period', len(period))

In [167]:
ref_period = nc.createVariable(varname= 'ref_period', datatype = 'i', dimensions='ref_period')
ref_median = nc.createVariable(varname= 'ref_median', datatype = 'float', dimensions='ref_period')
ref_mean = nc.createVariable(varname= 'ref_mean', datatype = 'float', dimensions='ref_period')
ref_std = nc.createVariable(varname= 'ref_std', datatype = 'float', dimensions='ref_period')

In [168]:
ref_period[:] = ref_periods
ref_median[:] = ref_medians
ref_mean[:] = ref_mean
ref_std[:] = ref_stds

In [ ]:
ref_period = nc.variables['ref_period']

In [169]:
ref_period_att = {'standard_name':'ref_period',
                  'long_name':'start year of reference period',
                  'units':'year' }
ref_median_att = {'standard_name':'ref_median',
                  'long_name':'median for reference period'}
ref_mean_att = {'standard_name':'ref_mean',
                  'long_name':'mean for reference period'}
ref_std_att = {'standard_name':'ref_std',
                  'long_name':'standart deviation for reference period'}

ref_period.setncatts(ref_period_att)
ref_std.setncatts(ref_std_att)
ref_mean.setncatts(ref_mean_att)
ref_median.setncatts(ref_median_att)

In [170]:
nc.close()
file

'/home/estimr2/nhempelmann/data/extremoscop/timeseries/R20mm/eobs/1/R20mm_EOBS_0.22_rot_v11_19500101-20141231.nc'

In [139]:
plt.plot(period,mean)
plt.plot(period,median)

plt.show()

In [39]:
std1 = hs['1961':'1990'].std()
std2 = hs['1971':'2000'].std()
std3 = hs['1976':'2005'].std()
print std1, std2, std3 

18.8503809238 19.3613986998 19.0443777937


In [40]:
median1 = hs['1961':'1990'].median()
median2 = hs['1971':'2000'].median()
median3 = hs['1976':'2005'].median()
print median1, median2, median3 

151.595237732 151.595237732 153.690475464


In [38]:
var1 = hs['1961':'1990'].var()
var2 = hs['1971':'2000'].var()
var3 = hs['1976':'2005'].var()
print var1, var2, var3 

355.336860972 374.863759612 362.688325549


In [70]:
nc.createDimension('ref_period', 35)

In [ ]:
ref_period = nc.createVariable(varname= 'ref_period', datatype = 'int', dimensions='ref_period')
ref_median = nc.createVariable(varname= 'ref_median', datatype = 'float', dimensions='ref_period')
ref_std = nc.createVariable(varname= 'ref_std', datatype = 'float', dimensions='ref_period')

In [62]:
ref_test = nc.createVariable(varname= 'ref_test', datatype = 'float', dimensions='ref_bnds')

In [ ]:
vals = [18.8503809238, 19.3613986998]
ref_test. = vals

In [78]:
nc.close()

In [ ]:
#cdo.setreftime('1949-12-01,00:00:00,days', input=geom_file, output='/home/nils/data/EOBS/tx_0.22deg_rot_2014_Cordex.nc')
#cdo.setname('tasmax', input='/home/nils/data/EOBS/tx_0.22deg_rot_2014_Cordex.nc' , output='/home/nils/data/EOBS/tasmax_EOBS-22_2014.nc')